In [4]:
import torch
torch.cuda.is_available()

AttributeError: module 'torch' has no attribute 'cuda'

In [2]:
from huggingface_hub import HfApi
available_pipelines = [p.modelId for p in HfApi().list_models(filter="pyannote-audio-pipeline")]
available_pipelines

['pyannote/voice-activity-detection',
 'pyannote/speaker-diarization',
 'pyannote/speaker-segmentation',
 'pyannote/overlapped-speech-detection']